# Analyzing GW170817

We will demonstrate how to use _jim_ to analyze the binary neutron star GW170817 using the IMRPhenomD waveform.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [30]:
import numpy as np
import jax.numpy as jnp
import jax

from gwpy.timeseries import TimeSeries
from gwpy.frequencyseries import FrequencySeries
import requests

from astropy.time import Time

from scipy.signal.windows import tukey
from scipy.interpolate import interp1d


from ripple.waveforms.IMRPhenomD import gen_IMRPhenomD_polar

from jaxgw.PE.detector_preset import *
from jaxgw.PE.heterodyneLikelihood import make_heterodyne_likelihood_mutliple_detector
from jaxgw.PE.detector_projection import make_detector_response

from flowMC.nfmodel.rqSpline import RQSpline
from flowMC.sampler.MALA import MALA
from flowMC.sampler.Sampler import Sampler
from flowMC.utils.PRNG_keys import initialize_rng_keys
from flowMC.nfmodel.utils import *

## Data and conditioning

We will fetch the GW170817 strain data recorded by LIGO and Virgo from [GWOSC](https://gw-openscience.org) using the [GWpy](https://gwpy.github.io) package; we will also download power-spectral densities (PSDs), made publicly available by LIGO-Virgo.

### Strain

To do so, we need to know the GPS time associated with the event (in this case, $t = 1187008882.43 s$).
We also need to prescribe how much data we wish to analyze around the event (in this case, $T = 128 s$, aka, the _segment length_ or _seglen_). We will place the trigger $2 s$ before the end of the analysis segment, following the LVK convention.

> &#x1F449; _**NOTE:** if you don't know the tigger GPS time, you may obtain it from the event name using the [`datasets.event_gps`](https://gwosc.readthedocs.io/en/stable/reference/gwosc.datasets.event_gps.html#event-gps) utility from the [gwosc](https://gwosc.readthedocs.io) package, e.g., `event_gps("GW170817")`_.


In [11]:
trigger_time = 1187008882.43
seglen = 128

# determine segment bounds, placing trigger 2s before the end
post_trigger_duration = 2
start = trigger_time - seglen + post_trigger_duration
end = trigger_time + post_trigger_duration

With those parameters, we can now fetch the data from GWOSC using `fetch_open_data()`. For GW170817, We make sure to specify `version=2` to get the version of data without the glitch in Livingston (see [GWOSC docs](https://doi.org/10.7935/K5B8566F) for this release).

In [13]:
ifos = ['H1', 'L1', 'V1']
data_td_dict = {i: TimeSeries.fetch_open_data(i, start, end, version=2)
                for i in ifos}

For the likelihood computation, we will want frequency domain data. We can IFFT the above data after applying a window function; following common LVK practice for this event, we apply a Tukey window with a slope parameter `alpha=0.00625`.

> &#x1F449; _**NOTE:** different `alpha` values may be appropriate for different events, e.g., `alpha = 0.4` is standard for shorter binary black holes._

In [49]:
tukey_alpha = 0.00625
data_fd_dict = {}
for ifo, d in data_td_dict.items():
    w = tukey(len(d), tukey_alpha)
    f = np.fft.rfftfreq(len(d), d=d.dt)
    data_fd_dict[ifo] = FrequencySeries(np.fft.rfft(d*w)/d.dt, frequencies=f)

### Power spectral densities (PSDs)

Besides the strain, to compute the likelihood we will need a PSDs characterizing the noise at each detector. Although we could estimate this oursevles directly from the data (e.g., [arXiv:1907.06540](https://arxiv.org/abs/1907.06540)), we will forgo that step and download precomputed PSDs made available by the LVK collaboration in [LIGO-P1800061](https://dcc.ligo.org/LIGO-P1800061/public).

> &#x1F449; _**NOTE:** you may load any PSD you wish for this step, whether from disk or computed on the fly._

In [28]:
psd_url = "https://dcc.ligo.org/public/0150/P1800061/011/GW170817_PSDs.dat"
with requests.get(psd_url) as r:
    psd_data = np.genfromtxt(r.iter_lines())

The `psd_data` object is a 2D array where the first column is frequency and the rest are the corresponding PSD values for H1, L1 and V1, in that order. For convenience, and because these PSD data are not uniformly sampled, we will turn this into interpolants that we can evaluate over any frequency bins for each detector.

In [53]:
psd_dict = {}
for i, (ifo, d) in enumerate(data_fd_dict.items()):
    p = interp1d(psd_data[:,0], psd_data[:,i+1], bounds_error=False,
                 fill_value=np.inf)
    psd_dict[ifo] = FrequencySeries(p(d.frequencies), frequencies=d.frequencies)

In [69]:
a = 1
b = 2

def test_func(x):
    """Test string {a}.

    This is a stest {b}.
    """
    return 2*x

test_func.__doc__ = test_func.__doc__.format(a=a, b=b)

In [71]:
test_func?

Signature: test_func(x)
Docstring:
Test string 1.

This is a stest 2.
File:      /tmp/ipykernel_2794160/1016926636.py
Type:      function

In [74]:
np.deg

<ufunc 'degrees'>